In [1]:
filenames = []

In [2]:
import nltk
import pandas as pd
import numpy as np
import re
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
K = 25


In [3]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/query.csv'
file = open(filename,'r')
text = re.sub('[0-9]+', '', file.read())
file.close()
queries = text.split('\n')
#queries = pd.read_csv(filename)
#queries.columns = ['No.','Text']

In [4]:
# import the common words to be used as stop words
filename = 'cacm/common_words'
file = open(filename,'r')
text = file.read()
file.close()
common_words = text.split('\n')


In [5]:

def createFileForArticle(article,count):
    file_name = 'cacm/cacm_seperated/cacm_' + str(count)
    lines = article.split('\n')
    new_file = open(file_name,'w+')
    filenames.append(file_name)
    for line in lines:     
       
        if(line.startswith('.X')):
            break
        if(not(line.startswith(('.T','.B','.A','.N','.X','.I')))):
            line_cleaned = re.sub('[0-9]+', '', line)
            line_cleaned = line_cleaned.lower()
            new_file.write(line_cleaned)
        

In [6]:
filename_cacm = 'cacm/cacm.all'
file = open(filename_cacm,'r')
text = file.read()
file.close()
count = 0
articles = text.split('.I ')
for article in articles:
    count+=1
    createFileForArticle(article,count)
    

In [7]:
# retrieve all files in cacm_seperated folder
import re
corpus=[]

for entry in filenames:
    file = open(str(entry),'rt')
    text = re.sub('[0-9]+', '', file.read())
    corpus.append(text.lower())
    file.close()


In [8]:
stopWords       = open ('cacm/common_words', 'rt').read()
stopWordList    = stopWords.split('\n')


In [9]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in tokenizer.tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            
            #remove tags
            token=re.sub("","",token)
    
            # remove special characters and digits
            token=re.sub("(\\d|\\W)+"," ",token)
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(stop_words='english',tokenizer=tokenize_and_stem,analyzer='word',max_df=0.9)
docs_tfidf = vectorizer.fit_transform(corpus)
filename = 'cacm/results/result.txt'
file = open(filename,'w+')

def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query,query_no):
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    sorted_similarity = sorted(((v, i+1) for i, v in enumerate(cosineSimilarities)), reverse=True)
    for t in sorted_similarity[:K]:
        #print(t[0])
        file.write(str(query_no) + " " +str(t[1]) + "\n")
    return cosineSimilarities

C:\Users\ZETTA\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [11]:
results = []

for index,query in enumerate(queries):
    similarity = get_tf_idf_query_similarity(vectorizer,docs_tfidf,query,index+1)
    sorted_similarity = sorted(((v, i+1) for i, v in enumerate(similarity)), reverse=True)
    #print(index+1,(sorted(similarity,reverse=True)[:3])
    #print(index+1,similarity)
    #results.append(index+1)
    #results.append(sorted_similarity[:1])
    #results.append((-similarity).argsort()[:5]+1)
    #results.append(sorted(similarity,reverse=True)[:3])

file.close()

Now it is time to measure accuracy and precision

In [12]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/qrels.text'
qrels = pd.read_csv(filename,sep="\s+")
qrels.columns = ['query', 'document','score','relevance']
qrels.drop(columns=['score','relevance'],axis=1,inplace=True)

In [13]:
#import the query relevance results and store it in a pandas datafrom
filename = 'cacm/results/result.txt'
preds = pd.read_csv(filename,sep="\s+")
preds.columns = ['query', 'document']

In [18]:
def get_precision(query_id):
    no_of_relevant_documents = qrels['query'].loc[qrels['query'] == query_id].count() or 1 
    no_of_documents_retrieved= K
    X = (preds['document'].loc[preds['query'] == query_id])
    y = (qrels['document'].loc[qrels['query'] == query_id])
    X_y_merged = pd.merge(X, y, how ='inner', on =['document'])
    no_of_relevant_documents_retrieved = X_y_merged['document'].count()
    #no_of_relevant_documents_retrieved = (preds.loc[preds['query'] == query_id])
    
    precision = no_of_relevant_documents_retrieved / K
    recall = no_of_relevant_documents_retrieved / no_of_relevant_documents
    return precision,recall


In [19]:
sum_precision = 0
sum_recall = 0
for query_id in range(1,len(queries)):
    p,r = get_precision(query_id)
    print(query_id,p,r)
    sum_precision+=p
    sum_recall+=r
print(sum_precision/52,sum_recall/52)

1 0.08 0.5
2 0.12 1.0
3 0.04 0.16666666666666666
4 0.12 0.25
5 0.08 0.25
6 0.12 1.0
7 0.36 0.32142857142857145
8 0.04 0.3333333333333333
9 0.2 0.5555555555555556
10 0.6 0.42857142857142855
11 0.4 0.5263157894736842
12 0.08 0.4
13 0.2 0.45454545454545453
14 0.64 0.36363636363636365
15 0.12 0.3
16 0.2 0.29411764705882354
17 0.2 0.3125
18 0.12 0.2727272727272727
19 0.32 0.7272727272727273
20 0.04 0.3333333333333333
21 0.04 0.09090909090909091
22 0.48 0.7058823529411765
23 0.04 0.25
24 0.08 0.15384615384615385
25 0.6 0.29411764705882354
26 0.36 0.3
27 0.48 0.41379310344827586
28 0.16 0.8
29 0.6 0.7894736842105263
30 0.08 0.5
31 0.08 1.0
32 0.08 0.6666666666666666
33 0.04 1.0
34 0.0 0.0
35 0.0 0.0
36 0.28 0.35
37 0.16 0.3333333333333333
38 0.4 0.625
39 0.32 0.6666666666666666
40 0.24 0.6
41 0.0 0.0
42 0.16 0.19047619047619047
43 0.32 0.1951219512195122
44 0.12 0.17647058823529413
45 0.44 0.4230769230769231
46 0.0 0.0
47 0.0 0.0
48 0.04 0.08333333333333333
49 0.16 0.5
50 0.0 0.0
51 0.0 0.0
5